In [1]:
###Pandas Libraries
import pandas as pd
import datetime
import time

#Dash Libraries
#import dash  # USE THIS IF RUNNING ON SERVER
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash # USE THIS IF RUNNING ON JUPYTER
import numpy as np

In [28]:
#*********                     FUNCTION RUNS ALL OF THE DATA MANIPULATION CODE
## TIMER FUNCTION
start = time.time()

mt_beds = 13 #Current Number of Beds Available in Maternity
ds_beds = 4  #Current Number of Beds Available in Delivery Suite

## Bring in Transfers data, update dates and remove uneeded columns
df = pd.read_csv('transfers.csv')
df = df.rename(columns = {'Start_Bed_Dttm':'Start', 'End_Bed_Dttm':'End'})
df = df.drop(columns=['Transfers', 'Bed_Code', 'Full Flow Code', 'Current Flow Code', '1st Level','PASID'])
date_cols = ['Start', 'End']
df[date_cols] = df[date_cols].apply(pd.to_datetime, format='%d/%m/%y %H:%M', errors='raise')
#df[date_cols] = df[date_cols].apply(lambda x: x.dt.strftime('%d/%m/%Y %H:%M:%S'))

#MATERNITY
## Make new dataframe patients entering Maternity Ward
mt_in_df= pd.DataFrame()
mt_in_df['Event'] = df.loc[df['Ward_Code'] == 'MT', 'Start']
mt_in_df['Patient_Change'] = 1
## make new dataframe with patients leaving room
mt_out_df= pd.DataFrame()
mt_out_df['Event'] = df.loc[df['Ward_Code'] == 'MT', 'End']
mt_out_df['Patient_Change'] = -1
## Concatenate the two datasets and sort by date to determin current patinet occupancies
mt_count_df = pd.concat([mt_in_df, mt_out_df])
mt_count_df.sort_values('Event', ascending = True, inplace=True)
mt_count_df['MT_Patients'] = mt_count_df['Patient_Change'].cumsum()
mt_count_df['Bed_Available'] = mt_count_df['MT_Patients'] <= mt_beds
mt_count_df.dropna(inplace = True)

# DELIVERY SUITE(DS)
## Make new dataframe patients entering room
ds_in_df = pd.DataFrame()
ds_in_df['Event'] = df.loc[df['Ward_Code'] == 'DS', 'Start']
ds_in_df['Patient_Change'] = 1
## make new dataframe with patients leaving room
ds_out_df= pd.DataFrame()
ds_out_df['Event'] = df.loc[df['Ward_Code'] == 'DS', 'End']
ds_out_df['Patient_Change'] = -1
## Concatenate the two datasets and sort by date to determin current patinet occupancies
ds_count_df = pd.concat([ds_in_df, ds_out_df])
ds_count_df.sort_values('Event', ascending = True, inplace=True)
ds_count_df['DS_Patients'] = ds_count_df['Patient_Change'].cumsum()
ds_count_df['Bed_Available'] = ds_count_df['DS_Patients'] <= mt_beds
ds_count_df.dropna(inplace = True)

# CLEAN DATA TO PRODUCE IDEAL PATIENT TRANSFERS
ideal_df = pd.DataFrame(df)
#CA, CL & HE Codes are converted to MT
ideal_df.loc[(ideal_df['Ward_Code'] == 'CA') | (ideal_df['Ward_Code'] == 'CL') | (ideal_df['Ward_Code'] == 'HE') , 'Ward_Code'] = 'MT' #Ward Code CA = MT
## Drop DPS from the new dataset
index_names = ideal_df.loc[(ideal_df['Ward_Code'] == 'DPS') | ((ideal_df['Ward_Code'] == 'ON'))].index
ideal_df.drop(index_names, inplace = True)
## Groupby the event code and with ward to summarise and finds any duplicates
events = ideal_df.groupby(['Link', 'Ward_Code'])
#events.get_group(('ADE-272865','MT'))
#get the min start date and max end date from the grouped results
ideal_df['min_start'] = events['Start'].transform('min')
ideal_df['max_end'] = events['End'].transform('max')
#Drop the Original start & end dates & rename the new calculated start & end
ideal_df.drop(columns=['Start', 'End'], inplace = True)
ideal_df.rename(columns= {'min_start':'Start', 'max_end': 'End'}, inplace = True)
ideal_df.sort_values(['Link', 'Ward_Code'], ascending = True, inplace = True)
ideal_df.drop_duplicates(keep='first', inplace = True)
#checks that there are no longer any duplicates
#ideal_df.loc[ideal_df.duplicated(keep=False), :]

# CALCULATE LENGTH OF STAY
ideal_df['LOS'] = (ideal_df['End'] - ideal_df['Start'])
ideal_df['LOS_hrs'] = ((ideal_df['End'] - ideal_df['Start']).dt.days * 24) + ((ideal_df['End'] - ideal_df['Start']).dt.seconds / (60*60))

# EXTRACT EVENTS DATA AND PUT IN THE IDEALISED DATA TABLE
events_df = pd.read_csv('exception_dates.csv')
#events_df.drop(columns=['Unnamed: 2'], inplace = True)
events_df.rename(columns = {'DATE':'Date', 'EVENT':'Event'}, inplace=True)
events_df['Date'] = events_df['Date'].apply(pd.to_datetime, format='%d/%m/%y', errors='raise')
events_df['Date'] = events_df['Date'].dt.date
events_df.sort_values(['Event', 'Date'])
## Combine all labels with the same date into the same field
grouped_events_df = events_df.groupby(['Date'])['Event'].apply(','.join).reset_index()
grouped_events_df.sort_values('Event', ascending=True)
## Delete erroneous Event Data
grouped_events_df = grouped_events_df.drop(grouped_events_df[grouped_events_df.Event == 'SCHOOL,SCHOOL'].index)
grouped_events_df = grouped_events_df.drop(grouped_events_df[grouped_events_df.Event == 'PUBLIC,SCHOOL,SCHOOL'].index)
## Merge data from the events table back into the Idealised data table
ideal_df['Date'] = ideal_df['Start'].dt.date
ideal_merged_df = pd.merge(grouped_events_df, ideal_df, how='right', on='Date')

end = time.time()
print(str(round(end - start,1)) + ' seconds to run code.')

0.4 seconds to run code.


In [31]:
#import dash  # USE THIS IF RUNNING ON SERVER
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash # USE THIS IF RUNNING ON JUPYTER
import numpy as np

#app = dash.Dash(__name__) # USE THIS IIF RUNNING ON SERVER
app = JupyterDash(__name__) # USE THIS IF RUNNING ON JUPYTER

app.layout = html.Div([
    
    html.Div([
        html.Pre(children= " Length of Stay Histogram Plot",
                style={"text-align": "center", "font-size": "100%", "color":"black"})
    ]),
    
    dcc.Dropdown(
        id='my_dropdown',
        options=[
            {'label': 'Maternity', 'value': 'MT'},
            {'label': 'Delivery Suite', 'value': 'DS'},
            {'label': 'ICU', 'value': 'ICU'},
                ],
                 optionHeight=35,
                 value='MT',
                 disabled=False,
                 multi=False,
                 searchable=True,
                 search_value='',
                 placeholder='Please select... ',
                 clearable=True
                 #style={'width':"100%"},
                 #className='select_box',
                 #persistence=True,
                 #persistence_type='memory' #'session','local' 
                ),
    
    dcc.Graph(id='the_graph')
    
])

@app.callback(
    Output(component_id ='the_graph', component_property='figure'), 
    [Input(component_id ='my_dropdown', component_property='value')])
def update_graph(my_dropdown):
    plot_df = ideal_df
    plot_df = plot_df.loc[plot_df['Ward_Code'] == my_dropdown, :]
    fig = px.histogram(plot_df, x="LOS_hrs", nbins=50)
    return fig

#app.run_server(debug=True)  # USE THIS IF RUNNING ON SERVER
app.run_server(mode='jupyterlab') # USE THIS IF RUNNING ON JUPYTER

In [29]:
plot_df = ideal_df
ideal_df
#plot_df['LOS'] = plot_df.LOS.dt.seconds / (60*60)
#plot_df

,Link,Ward_Code,Start,End,LOS,LOS_hrs,Date
1,ADE-271165,MT,2013-06-30 22:21:00,2013-07-04 10:40:00,3 days 12:19:00,84.316667,2013-06-30
4,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00,0 days 21:40:00,21.666667,2013-06-27
5,ADE-271206,MT,2013-06-28 10:20:00,2013-07-03 09:59:00,4 days 23:39:00,119.650000,2013-06-28
12,ADE-271551,MT,2013-06-27 23:54:00,2013-07-01 09:25:00,3 days 09:31:00,81.516667,2013-06-27
14,ADE-271568,DS,2013-07-11 07:16:00,2013-07-12 13:59:00,1 days 06:43:00,30.716667,2013-07-11
...,...,...,...,...,...,...,...
2027,ADE-510071,DS,2020-07-18 09:51:00,2020-07-19 11:45:00,1 days 01:54:00,25.900000,2020-07-18
2029,ADE-510071,MT,2020-07-19 11:45:00,2020-07-23 11:50:00,4 days 00:05:00,96.083333,2020-07-19
1522,ADE-510160,DS,2020-07-20 22:00:00,2020-07-21 13:15:00,0 days 15:15:00,15.250000,2020-07-20
3060,ADE-510160,MT,2020-07-21 15:20:00,2020-07-25 10:45:00,3 days 19:25:00,91.416667,2020-07-21


# Maternity Service Capacity - Model, Simulation & Analysis
## Objective 1
**Model current Maternity Service activity using process mapping and queuing network analysis**
1. Data Required
2. Profile - Patient Process Flow - Historical
3. Profile - Patient Process Flow - Idealised
4. Profile - Length of Stay
5. Profile - Arrival Rate
7. Variability Analysis - Events of Significance
8. Variability Analysis - Patient Process Flow
9. Variability Analysis - Length of Stay
10. Variability Analysis - Arrival Rate


## Objective 2
**Determine current capacity requirements using simulation and mathematical modeling**
1. Current Capacity Requirements - Historical Data
2. Current Capacity Requirements - Idealised Historical Data
3. Current Capacity Requirements - Simulated Model
4. Results Analysis - Accuracy with Idealised Historical Data

## Objective 3
**Determine future capacity requirements at 5 and 10 years based on population demographics projections data.**
1. Capacity Requirements - 5 year forecast
2. Capacity Requirements - 10 year forecast

***

## Objective 1
### 1.1. Data Required
#### 1.1.1 Historical Data
7 years of patient transfers data provided by hospital.
Data will provide the foundation of the mathematical model & data analysis

In [ ]:
df

#### 1.1.2  Critical Capacity Data
Key data to hospital Capacity Analysis

In [ ]:
mt_beds = 13 #Current Number of Beds Available in Maternity
ds_beds = 4  #Current Number of Beds Available in Delivery Suite

### 1.4 Profile Length of Stay

#### 1.4.2 Profile of Length of Stay by Ward

In [ ]:
ideal_df.groupby('Ward_Code').describe()

#### 1.4.2 Profile of Length of Stay Maternity

In [ ]:
ideal_df['LOS'][ideal_df['Ward_Code'] == 'MT'].value_counts(bins=10, normalize=True)*100

#### 1.4.3 Profile of Length of Stay Maternity

In [ ]:
ideal_df['LOS'][ideal_df['Ward_Code'] == 'DS'].value_counts(bins=10, normalize=True)*100

### 1.6 Variability Analysis - Events of Significance
#### 1.6.1 Selected Events & Associated Dates

In [ ]:
grouped_events_df.set_index('Date', inplace = True)
grouped_events_df.sample(frac=0.1)

#### 1.6.2 Number of days in Dataset that these dates occur

In [ ]:
grouped_events_df['Count'] = 1
grouped_events_df.groupby(['Event'], as_index=False)['Count'].count().sort_values('Count', ascending = False)

#### 1.6.3 Events of Significance brought back into idealised data

In [ ]:
ideal_merged_df['Event'].fillna('NORMAL', inplace=True)
ideal_merged_df
#ideal_merged_df['Event'].notnull().sum()

## Objective 2
### 2.1 Current Capacity Requirements - Historical Data
#### 2.1.1 Maternity Transformed Data
Historical data of number of patients in Maternity at the same time

In [ ]:
mt_count_df

#### 2.1.2 Maternity Occupancy Profile

In [ ]:
mt_count_df['MT_Patients'].describe()

#### 2.1.3 Delivery Suite Transformed Data

In [ ]:
ds_count_df

#### 2.1.4 Delivery Suite Occupancy Profile

In [ ]:
ds_count_df['DS_Patients'].describe()

### 2.2 Current Capacity Requirements - Idealised Historical Data
**Create new ideal patient flow records with existing data and clean the dataset**

In an ideal scenario, rather than sending patients to St Clares, St Helens and St Catherines they are sent to Maternity.  This process adjusts the actual data to what the ideal data would be.

As DPS is not relevant in this study we would also like to remove these entries from the dataset.

Once the data has been converted to ideal there are entries where the patient is transferred to the same room they left.  To clean the data we need to find the start date of when they first arrvied in the room and the end date when they left.

Once found we need to remove the duplicate entries and only show the first start date and last end date.

In [ ]:
ideal_df

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/markaldred/.pyenv/versions/3.9.0/lib/python3.9/threading.py", line 950, in _bootstrap_inner
    self.run()
  File "/Users/markaldred/.pyenv/versions/3.9.0/lib/python3.9/threading.py", line 888, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/markaldred/developer/dash_app/venv/lib/python3.9/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/Users/markaldred/developer/dash_app/venv/lib/python3.9/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/Users/markaldred/developer/dash_app/venv/lib/python3.9/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/Users/markaldred/developer/dash_app/venv/lib/python3.9/site-packages/six.py", line 703, in reraise
    raise value
  File "/Users/markaldred/developer/dash_app/venv/lib/python3.9/site-packag